In [50]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI(openai_api_key=api_key,temperature=0.1,model="gpt-4o-mini")

cache_dir = LocalFileStore("./.cache/")


splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    separators=["\n\n", ".", "?", "!"],
    chunk_size=1000,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

memory = ConversationBufferMemory(
    llm=llm,
    memory_key="history",
    return_messages=True
    )

def load_memory(_):
        return memory.load_memory_variables({})["history"]


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 문서를 읽고 정보를 전달해주는 AI 입니다. 주어지는 문서를 통해 질문에 대한 대답을 해주세요. 만약 문서에 정보를 찾지 못한다면, 직접 답변을 새로운 것을 만들어 내지 말고 '모르겠습니다.' 라고 답변하면서 그 이유를 알려주세요. 대답은 한국어로 번역해서 전달 해주세요. :\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
        {
                "context": retriver,
                "question": RunnablePassthrough(),
                "history": RunnableLambda(load_memory),
         }
         | prompt
         | llm
         )

def invoke_chain(question):
        result = chain.invoke(question).content
        memory.save_context(
                {"input":question},
                {"output":result}
        )
        print(result)


In [51]:
invoke_chain("Is Aaronson guilty?")

모르겠습니다. 문서에서는 Aaronson이 범죄로 기소되었다고 언급되지만, 그의 실제 유죄 여부에 대한 정보는 제공되지 않습니다.


In [52]:
invoke_chain("What message did he write in the table?")

그가 테이블에 쓴 메시지는 다음과 같습니다:

"FREEDOM IS SLAVERY" (자유는 노예다)

그 아래에는 "TWO AND TWO MAKE FIVE" (둘과 둘은 다섯이다)라는 문구도 적었습니다.


In [53]:
invoke_chain("Who is Julia?")

문서에서는 Julia가 Winston의 사랑하는 사람으로 언급됩니다. 그녀는 Winston과 함께 자유롭게 지내던 시절에 그와 깊은 관계를 맺었으나, 이후 상황이 변하면서 그들의 관계도 복잡해졌습니다. Julia는 Winston이 감정적으로 의지하는 인물이며, 그에 대한 사랑과 배신의 감정이 얽혀 있습니다.
